### Imports & Settings

In [49]:
## Imports

# From standard library
import json
import logging
import time
from typing import Tuple

# From third party
import boto3
import botocore
from dotenv import dotenv_values
import pandas as pd
import psycopg2

In [2]:
# Logging configuration
logging.basicConfig(
    filename="./project.log",
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    level=logging.INFO,
)

In [3]:
# Overall region
REGION_NAME = "us-west-2"

# Policies required
REQUIRED_POLICIES = ["AmazonRedshiftFullAccess", "AmazonS3ReadOnlyAccess", "IAMFullAccess"]

# IAM Role Global Variables
ROLE_NAME = "dwhRole"
ASSUME_ROLE_POLICY_DOCUMENT = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": "sts:AssumeRole",
            "Effect": "Allow",
            "Principal": {"Service": "redshift.amazonaws.com"},
        }
    ],
}
POLICY_ARN = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"  # For attaching policies

# Redshift Global Variables
CLUSTER_IDENTIFIER = "dwhCluster"
CLUSTER_TYPE = "multi-node"
NUMBER_OF_NODES = 4
NODE_TYPE = "dc2.large"
DB_NAME = "dwh"
MASTER_USER_NAME = "dwhuser"
MASTER_USER_PASSWORD = "Passw0rd"

# Connection Global Variables
PORT = 5439
CIDR_IP = "0.0.0.0/0"
IP_PROTOCOL = "TCP"

# S3 Global Variables
LOG_DATA = "s3://udacity-dend/log_data"
LOG_JSONPATH = "s3://udacity-dend/log_json_path.json"
SONG_DATA = "s3://udacity-dend/song_data"

In [4]:
# Get AWS credentials from .env file
env = dotenv_values()

AWS_ACCESS_KEY_ID = env["AWS_ACCESS_KEY_ID"]
AWS_SECRET_ACCESS_KEY = env["AWS_SECRET_ACCESS_KEY"]

### Create Clients

In [5]:
def create_clients(
        aws_access_key_id: str, 
        aws_secret_access_key: str, 
        region_name: str = REGION_NAME
    ) -> Tuple[botocore.client, ...]:
    """Creates the required clients for the project."""
    try:
        sts = boto3.client(
            "sts",
            region_name=region_name,
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
        )
        iam = boto3.client(
            "iam",
            region_name=region_name,
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
        )
        s3 = boto3.client(
            "s3",
            region_name=region_name,
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
        )
        redshift = boto3.client(
            "redshift",
            region_name=region_name,
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
        )
        ec2 = boto3.client(
            "ec2",
            region_name=region_name,
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
        )
        return sts, iam, s3, redshift, ec2
    except Exception as e:
        logging.error(e)
        raise e

In [13]:
sts, iam, s3, redshift, ec2 = create_clients(AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, region_name=REGION_NAME)
sts, iam, s3, redshift, ec2

(<botocore.client.STS at 0x113d993d0>,
 <botocore.client.EC2 at 0x113f6e410>)

### S3 Exploration

In [27]:
def bucket_name_and_prefix_from_string(bucket_string: str) -> Tuple[str, str]:
    """Creates a filter for the bucket."""
    bucket_name = bucket_string.split("/")[2]
    prefix = f"{'/'.join(bucket_string.split('/')[3:])}/"
    return bucket_name, prefix

In [42]:
log_data = []

bucket_name, prefix = bucket_name_and_prefix_from_string(LOG_DATA)
bucket_name, prefix

paginator = s3.get_paginator("list_objects")
page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=prefix)

for page in page_iterator:
    for item in page["Contents"]:
        key = item["Key"]
        if key != prefix:
            object_body = s3.get_object(Bucket=bucket_name, Key=key)["Body"].read().decode("utf-8")
            for data in object_body.split("\n"):
                if data:
                    log_data.append(json.loads(data))

In [47]:
song_data = []

bucket_name, prefix = bucket_name_and_prefix_from_string(SONG_DATA)
bucket_name, prefix

paginator = s3.get_paginator("list_objects")
page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=prefix)

for page in page_iterator:
    for item in page["Contents"]:
        key = item["Key"]
        if key != prefix:
            object_body = s3.get_object(Bucket=bucket_name, Key=key)["Body"].read().decode("utf-8")
            for data in object_body.split("\n"):
                if data:
                    song_data.append(json.loads(data))

In [48]:
len(log_data), len(song_data)

(8056, 14896)

In [57]:
log_data[4_000]

{'artist': 'Sound 5',
 'auth': 'Logged In',
 'firstName': 'Jacob',
 'gender': 'M',
 'itemInSession': 2,
 'lastName': 'Klein',
 'length': 451.21261,
 'level': 'paid',
 'location': 'Tampa-St. Petersburg-Clearwater, FL',
 'method': 'PUT',
 'page': 'NextSong',
 'registration': 1540558108796.0,
 'sessionId': 518,
 'song': 'Latin Static',
 'status': 200,
 'ts': 1542462343796,
 'userAgent': '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"',
 'userId': '73'}

In [50]:
log_df = pd.DataFrame(log_data)
log_df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1.540919e+12,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1.540345e+12,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1.540345e+12,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8051,Timbiriche,Logged In,Rylan,M,58,George,202.60526,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,Besos De Ceniza,200,1543603476796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
8052,A Perfect Circle,Logged In,Rylan,M,59,George,206.05342,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,Rose,200,1543603678796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
8053,Anberlin,Logged In,Rylan,M,60,George,348.68200,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,The Haunting,200,1543603884796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
8054,None,Logged In,Rylan,M,61,George,NaN,paid,"Birmingham-Hoover, AL",GET,Downgrade,1.541020e+12,1076,None,200,1543603993796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16


In [55]:
log_df.to_csv("log_data.csv", index=False)

In [58]:
song_data[4_000]

{'artist_id': 'ARLYGIM1187FB4376E',
 'artist_latitude': None,
 'artist_location': '',
 'artist_longitude': None,
 'artist_name': 'Joe Higgs',
 'duration': 162.82077,
 'num_songs': 1,
 'song_id': 'SOQOOPI12A8C13B040',
 'title': 'Wake up And Live',
 'year': 1975}

In [51]:
song_df = pd.DataFrame(song_data)
song_df

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARJNIUY12298900C91,NaN,,NaN,Adelitas Way,213.94240,1,SOBLFFE12AF72AA5BA,Scream,2009
1,AR73AIO1187B9AD57B,37.77916,"San Francisco, CA",-122.42005,Western Addiction,118.07302,1,SOQPWCR12A6D4FB2A3,A Poor Recipe For Civic Cohesion,2005
2,ARMJAGH1187FB546F3,35.14968,"Memphis, TN",-90.04892,The Box Tops,148.03546,1,SOCIWDW12A8C13D406,Soul Deep,1969
3,AR9Q9YC1187FB5609B,NaN,New Jersey,NaN,Quest_ Pup_ Kevo,252.94322,1,SOFRDWL12A58A7CEF7,Hit Da Scene,0
4,ARSVTNL1187B992A91,51.50632,"London, England",-0.12714,Jonathan King,129.85424,1,SOEKAZG12AB018837E,I'll Slap Your Face (Entertainment USA Theme),2001
...,...,...,...,...,...,...,...,...,...,...
14891,ARANAED11F50C473A4,NaN,,NaN,Lil Wyte,39.47057,1,SOZTJBR12A8C142681,Death & Life (Skit),0
14892,AR3WYZ21187B9B4F3F,47.03922,"Olympia, WA",-122.89143,Bikini Kill,141.73995,1,SOAKQBT12AB01803CB,Sugar,1993
14893,ARYJ6RP1187B9B3827,36.16778,"Nashville, TN",-86.77836,Cindy Morgan,287.05914,1,SOOXNKG12A58A7C6BC,Someone Believes In You - Album Version,0
14894,AREFNKX1187B991576,40.71455,NY - New York City,-74.00712,Dan Zanes,172.98240,1,SOCCODE12A8C136A89,We Shall Not Be Moved,0


In [54]:
song_df.to_csv("song_data.csv", index=False)

### Check User

In [ ]:
def get_user_name(
        sts: botocore.client
    ) -> str:
    """Get the user name from STS client."""
    try:
        user_info = sts.get_caller_identity()
        return user_info["Arn"].split("/")[1]
    except Exception as e:
        logging.error(e)
        raise e

In [ ]:
user_name = get_user_name(sts)
user_name

In [ ]:
def user_has_required_policies(
        iam: botocore.client,
        user_name: str,
        required_policies: list = REQUIRED_POLICIES
    ) -> bool:
    """Check if the user has the required policies from IAM client."""
    try:
        user_policies = iam.list_attached_user_policies(UserName=user_name)["AttachedPolicies"]
        user_policies = [policy["PolicyName"] for policy in user_policies]
        return all(policy in user_policies for policy in required_policies)
    except Exception as e:
        logging.error(e)
        raise e

In [ ]:
user_has_required_policies(iam, user_name=user_name, required_policies=REQUIRED_POLICIES)

### Role Handling

In [ ]:
def role_exists(
    iam: botocore.client, 
    role_name: str = ROLE_NAME
) -> bool:
    """Check if the role exists from IAM client."""
    try:
        iam.get_role(RoleName=role_name)
        return True
    except iam.exceptions.NoSuchEntityException:
        return False
    except botocore.exceptions.ClientError as e:
        logging.error(e)
        raise e

In [ ]:
role_exists(iam, role_name=ROLE_NAME)

In [ ]:
def create_role(
        iam: botocore.client, 
        role_name: str = ROLE_NAME, 
        assume_role_policy_document: str = ASSUME_ROLE_POLICY_DOCUMENT
    ) -> None:
    """Create the role from IAM client."""
    if not role_exists(iam, role_name):
        try:
            iam.create_role(
                RoleName=role_name,
                AssumeRolePolicyDocument=json.dumps(assume_role_policy_document),
            )
            waiter = iam.get_waiter("role_exists")
            waiter.wait(RoleName=role_name)
            assert role_exists(iam, role_name), "Something went wrong. Role was not created."
        except Exception as e:
            logging.error(e)
            raise e

In [ ]:
create_role(iam)

In [ ]:
role_exists(iam)

In [ ]:
def role_assumes_relavant_role_policy_document(
    iam: botocore.client,
    role_name: str = ROLE_NAME, 
    assume_role_policy_document: dict = ASSUME_ROLE_POLICY_DOCUMENT
) -> bool:
    """Check if the role has the right required trust relationship from IAM client."""
    if role_exists(iam, role_name):
        try:
            role = iam.get_role(RoleName=role_name)
            return role["Role"]["AssumeRolePolicyDocument"] == assume_role_policy_document
        except Exception as e:
            logging.error(e)
            raise e
    else:
        return False

In [ ]:
role_assumes_relavant_role_policy_document(iam, role_name=ROLE_NAME, assume_role_policy_document=ASSUME_ROLE_POLICY_DOCUMENT)

In [ ]:
def role_has_required_policy_attached(
    iam: botocore.client,
    role_name: str = ROLE_NAME, 
    policy_arn: str = POLICY_ARN
) -> bool:
    """Check if the role has the required policy attached from IAM client."""
    if role_exists(iam, role_name):
        try:
            role_policies = iam.list_attached_role_policies(RoleName=role_name)["AttachedPolicies"]
            role_policies = [policy["PolicyArn"] for policy in role_policies]
            return policy_arn in role_policies
        except Exception as e:
            logging.error(e)
            raise e
    else:
        return False

In [ ]:
role_has_required_policy_attached(iam, role_name=ROLE_NAME, policy_arn=POLICY_ARN)

In [ ]:
def attach_required_policy_to_role(
    iam: botocore.client, 
    role_name: str = ROLE_NAME, 
    policy_arn: str = POLICY_ARN
    ) -> None:
    """Attach the required policy to the user from IAM client."""
    if role_exists(iam, role_name) and not role_has_required_policy_attached(iam, role_name, policy_arn):
        try:
            iam.attach_role_policy(RoleName=role_name, PolicyArn=policy_arn)
            # time.sleep(10)
            assert role_has_required_policy_attached(iam, role_name, policy_arn), "Something went wrong, Policy is not attached."
        except Exception as e:
            logging.error(e)
            raise e

In [ ]:
attach_required_policy_to_role(iam, role_name=ROLE_NAME, policy_arn=POLICY_ARN)

In [ ]:
role_has_required_policy_attached(iam, ROLE_NAME, POLICY_ARN)

In [ ]:
def role_completed(
        iam: botocore.client, 
        role_name: str = ROLE_NAME, 
        assume_role_policy_document: dict = ASSUME_ROLE_POLICY_DOCUMENT, 
        policy_arn: str = POLICY_ARN
    ) -> bool:
    """Check if the role is completed from IAM client."""
    return (
        role_exists(iam, role_name) & 
        role_assumes_relavant_role_policy_document(iam, role_name, assume_role_policy_document) & 
        role_has_required_policy_attached(iam, role_name, policy_arn)
    )

In [ ]:
role_completed(iam, role_name=ROLE_NAME, assume_role_policy_document=ASSUME_ROLE_POLICY_DOCUMENT, policy_arn=POLICY_ARN)

In [ ]:
def build_role_as_necessary(
    iam: botocore.client, 
    role_name: str = ROLE_NAME, 
    assume_role_policy_document: 
    dict = ASSUME_ROLE_POLICY_DOCUMENT, 
    policy_arn: str = POLICY_ARN
) -> str:
    """Builds the role and the required compenents 
    if not already existing from IAM client and returns the ARN."""
    if not role_exists(iam, role_name):
        create_role(iam, role_name, assume_role_policy_document)
    if not role_assumes_relavant_role_policy_document(iam, role_name, assume_role_policy_document):
        error_message = "Role already exists but with a different trust relationship. Please visit the AWS console to amend the policy."
        logging.error(error_message)
        raise Exception(error_message)
    if not role_has_required_policy_attached(iam, role_name, policy_arn):
        attach_required_policy_to_role(iam, role_name, policy_arn)
    assert role_completed(iam, role_name, assume_role_policy_document, policy_arn), "Something went wrong. The role is not completed."
    return iam.get_role(RoleName=role_name)["Role"]["Arn"]

In [ ]:
iam_roles = build_role_as_necessary(iam, role_name=ROLE_NAME, assume_role_policy_document=ASSUME_ROLE_POLICY_DOCUMENT, policy_arn=POLICY_ARN)
iam_roles

In [ ]:
def destroy_role(
    iam: botocore.client, 
    role_name: str = ROLE_NAME
) -> None:
    """Destroys the role from IAM client by detaching all policies and deleting the role."""
    if role_exists(iam, role_name):
        try:
            role_policies = iam.list_attached_role_policies(RoleName=role_name)["AttachedPolicies"]
            for policy in role_policies:
                iam.detach_role_policy(RoleName=role_name, PolicyArn=policy["PolicyArn"])
            iam.delete_role(RoleName=role_name)
            assert not role_exists(iam, role_name), "Something went wrong. The role still exists."
        except Exception as e:
            logging.error(e)
            raise e

In [ ]:
destroy_role(iam)

In [ ]:
role_exists(iam, role_name=ROLE_NAME)

In [ ]:
iam_roles = build_role_as_necessary(iam, role_name=ROLE_NAME, assume_role_policy_document=ASSUME_ROLE_POLICY_DOCUMENT, policy_arn=POLICY_ARN)
iam_roles

### Redshift Handling

In [ ]:
def redshift_cluster_exists(redshift: botocore.client, cluster_identifier: str = CLUSTER_IDENTIFIER) -> bool:
    """Check if the cluster exists from Redshift client."""
    try:
        clusters = redshift.describe_clusters(ClusterIdentifier=cluster_identifier)["Clusters"]
        return len(clusters) == 1
    except redshift.exceptions.ClusterNotFoundFault as e:
        return False
    except Exception as e:
        logging.error(e)
        raise e

In [ ]:
redshift_cluster_exists(redshift, cluster_identifier=CLUSTER_IDENTIFIER)

In [ ]:
def create_redshift_cluster(
    redshift: botocore.client, 
    iam_roles: str, 
    cluster_identifier: str = CLUSTER_IDENTIFIER, 
    cluster_type: str = CLUSTER_TYPE, 
    node_type: str = NODE_TYPE, 
    number_of_nodes: int = NUMBER_OF_NODES, 
    db_name: str = DB_NAME, 
    master_user_name: str = MASTER_USER_NAME,
    master_user_password: str = MASTER_USER_PASSWORD, 
) -> str:
    """Creates the cluster from Redshift client."""
    try:
        redshift.create_cluster(
            IamRoles=[iam_roles],
            ClusterIdentifier=cluster_identifier, 
            ClusterType=cluster_type, 
            NodeType=node_type, 
            NumberOfNodes=number_of_nodes, 
            DBName=db_name, 
            MasterUsername=master_user_name, 
            MasterUserPassword=master_user_password, 
        )
        waiter = redshift.get_waiter("cluster_available")
        waiter.wait(ClusterIdentifier=cluster_identifier)
        assert redshift_cluster_exists(redshift, cluster_identifier), "Something went wrong. The cluster does not exist."
        return redshift.describe_clusters(ClusterIdentifier=cluster_identifier)["Clusters"][0]["VpcSecurityGroups"][0]["VpcSecurityGroupId"]
    except redshift.exceptions.ClusterAlreadyExistsFault as e:
        return redshift.describe_clusters(ClusterIdentifier=cluster_identifier)["Clusters"][0]["VpcSecurityGroups"][0]["VpcSecurityGroupId"]
    except Exception as e:
        logging.error(e)
        raise e 

In [ ]:
group_id = create_redshift_cluster(redshift, iam_roles)
group_id

In [ ]:
redshift_cluster_exists(redshift, cluster_identifier=CLUSTER_IDENTIFIER)

In [ ]:
def redshift_cluster_is_available(
    redshift: botocore.client,
    cluster_identifier: str = CLUSTER_IDENTIFIER
) -> bool:
    """Check if the cluster is available from Redshift client."""
    if redshift_cluster_exists(redshift, cluster_identifier):
        try:
            cluster_status = redshift.describe_clusters(ClusterIdentifier=cluster_identifier)["Clusters"][0]["ClusterStatus"]
            return cluster_status == "available"
        except Exception as e:
            logging.error(e)
            raise e

In [ ]:
redshift_cluster_is_available(redshift, cluster_identifier=CLUSTER_IDENTIFIER)

In [ ]:
def cluster_is_complete(
    redshift: botocore.client, 
    iam_roles: str,
    cluster_identifier: str = CLUSTER_IDENTIFIER,
    node_type: str = NODE_TYPE,
    number_of_nodes: int = NUMBER_OF_NODES,
    db_name: str = DB_NAME,
    master_user_name: str = MASTER_USER_NAME,
    # port: int = PORT
) -> bool:
    """Check if the cluster is complete from Redshift client."""
    try:
        if not redshift_cluster_exists(redshift, cluster_identifier):
            return False
        elif not redshift_cluster_is_available(redshift, cluster_identifier):
            return False
        else:
            cluster = redshift.describe_clusters(ClusterIdentifier=cluster_identifier)["Clusters"][0]
            return (
                cluster["NodeType"] == node_type and 
                cluster["NumberOfNodes"] == number_of_nodes and 
                cluster["IamRoles"][0]["IamRoleArn"] == iam_roles and
                cluster["DBName"] == db_name and
                cluster["MasterUsername"] == master_user_name 
            )
    except Exception as e:
        logging.error(e)
        raise e

In [ ]:
cluster_is_complete(redshift, iam_roles, cluster_identifier=CLUSTER_IDENTIFIER, node_type=NODE_TYPE, number_of_nodes=NUMBER_OF_NODES, db_name=DB_NAME, master_user_name=MASTER_USER_NAME)#, port=PORT)

In [ ]:
def get_redshift_host(redshift: botocore.client, cluster_identifier: str = CLUSTER_IDENTIFIER) -> str:
    """Get the host from Redshift client."""
    try:
        return redshift.describe_clusters(ClusterIdentifier=cluster_identifier)["Clusters"][0]["Endpoint"]["Address"]
    except Exception as e:
        logging.error(e)
        raise e

In [ ]:
host = get_redshift_host(redshift, cluster_identifier=CLUSTER_IDENTIFIER)
host

In [ ]:
def authorize_ingress(
    ec2: botocore.client, 
    group_id: str, 
    cidr_ip: str = CIDR_IP,
    ip_protocol: str = IP_PROTOCOL,
    port: int = PORT
) -> None:
    """Authorizes the ingress from EC2 client."""
    try:
        ec2.authorize_security_group_ingress(
            GroupId=group_id,
            CidrIp=cidr_ip,
            IpProtocol=ip_protocol,
            FromPort=port,
            ToPort=port,
            )
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == 'InvalidPermission.Duplicate':
            print('The rule already exists.')
        else:
            logging.error(e)
            raise e
    except Exception as e:
        logging.error(e)
        raise e

In [ ]:
authorize_ingress(ec2, group_id, cidr_ip=CIDR_IP, ip_protocol=IP_PROTOCOL, port=PORT)

In [ ]:
def ingress_is_authorized(
    ec2: botocore.client, 
    group_id: str, 
    cidr_ip: str = CIDR_IP,
    ip_protocol: str = IP_PROTOCOL,
    port: int = PORT
) -> bool:
    """Check if the ingress is authorized from EC2 client."""
    try:
        ip_permission = ec2.describe_security_groups(GroupIds=[group_id])["SecurityGroups"][0]["IpPermissions"][0]
        if ip_permission.get("IpRanges") == []:
            return False
        else:
            return all([
                ip_permission["IpProtocol"] == ip_protocol.lower(), 
                ip_permission["IpRanges"][0]["CidrIp"] == cidr_ip if ip_permission["IpRanges"] != [] else False,
                ip_permission["IpRanges"][0]["CidrIp"] == cidr_ip,
                ip_permission["FromPort"] == port,
                ip_permission["ToPort"] == port,
            ])
    except Exception as e:
        logging.error(e)
        raise e

In [ ]:
ingress_is_authorized(ec2, group_id, cidr_ip=CIDR_IP, ip_protocol=IP_PROTOCOL, port=PORT)

##### Check Database Connection

In [ ]:
configuration = {
    "host": host,
    "port": PORT,
    "database": DB_NAME,
    "user": MASTER_USER_NAME,
    "password": MASTER_USER_PASSWORD,
}

connection = psycopg2.connect(**configuration)
cursor = connection.cursor()
cursor.connection.encoding

In [ ]:
def revoke_ingress(ec2: botocore.client, group_id: str, cidr_ip: str = CIDR_IP, ip_protocol: str = IP_PROTOCOL, port: int = PORT) -> None:
    """Revokes the ingress from EC2 client."""
    try:
        ec2.revoke_security_group_ingress(
            GroupId=group_id,
            IpProtocol=ip_protocol,
            CidrIp=cidr_ip,
            FromPort=port,
            ToPort=port,
            )
    except Exception as e:
        logging.error(e)
        raise e        

In [ ]:
revoke_ingress(ec2, group_id, cidr_ip=CIDR_IP, ip_protocol=IP_PROTOCOL, port=PORT)

In [ ]:
ingress_is_authorized(ec2, group_id, cidr_ip=CIDR_IP, ip_protocol=IP_PROTOCOL, port=PORT)

In [ ]:
def delete_cluster(redshift: botocore.client, cluster_identifier: str = CLUSTER_IDENTIFIER) -> None:
    try:
        redshift.delete_cluster(ClusterIdentifier=CLUSTER_IDENTIFIER, SkipFinalClusterSnapshot=True)
        waiter = redshift.get_waiter("cluster_deleted")
        waiter.wait(ClusterIdentifier=CLUSTER_IDENTIFIER)
    except botocore.exceptions.ClientError as e:
        if e.response["Error"]["Code"] == "ClusterNotFound":
            print("Cluster not found.")
        else:
            logging.error(e)
            raise e
    except Exception as e:
        logging.error(e)
        raise e

In [ ]:
delete_cluster(redshift, cluster_identifier=CLUSTER_IDENTIFIER)

In [ ]:
destroy_role(iam, role_name=ROLE_NAME)